# HW 10 CLUSTERING BUSINESS

# cluster time trends in NYC businesses: 
# DATA
# Census Business data:
## download census data for businesses by ZIP code. the data is here
http://www.census.gov/econ/cbp/download/
##  you can  download it with 3 terminal commands as follows: the data from 1993 through 2001 is different in the format of its path than the data after 2001 (that is why more than one for loop is needed)



NOTES: 
to read in a zip file without unzipping it you can use the pandas and zipfile packages

In [1]:
import pandas as pd
import geopandas as  gpd
import os
import zipfile #i am not giving it a name cause i intend to use it only once
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

%matplotlib inline

### 1.get and prepare data
Geojason data is in my folder PUIdata/Census Business data, called 'nyc-zip-code-tabulation-areas-polygons.geojson'

Census data is in my folder PUIdata/Census Business data, called 'zbpXXtotals.zip' (XX = 94 ~ 15)



In [2]:
path = os.getenv("PUIDATA")+ '/Census Business data/'
path

'/home/cusp/cc5219/PUIdata/Census Business data/'

In [12]:
#geo data
geonyc = gpd.read_file(path + "nyc-zip-code-tabulation-areas-polygons.geojson")


In [13]:
geonyc.head()

,@id,BLDGpostalCode,CTY_FIPS,OBJECTID,PO_NAME,STATE,ST_FIPS,Shape_Area,Shape_Leng,borough,geometry,postalCode
0,http://nyc.pediacities.com/Resource/PostalCode...,0,081,1,Jackson Heights,NY,36,2.016328e+07,20624.692317,Queens,POLYGON ((-73.86942457284175 40.74915687096787...,11372
1,http://nyc.pediacities.com/Resource/PostalCode...,0,081,2,Glen Oaks,NY,36,2.260653e+07,23002.816039,Queens,POLYGON ((-73.71068374074007 40.75004039450917...,11004
2,http://nyc.pediacities.com/Resource/PostalCode...,0,081,3,New Hyde Park,NY,36,6.269333e+06,15749.161511,Queens,POLYGON ((-73.70098278625547 40.73889569923034...,11040
3,http://nyc.pediacities.com/Resource/PostalCode...,0,081,4,Bellerose,NY,36,4.941836e+07,35932.810639,Queens,POLYGON ((-73.72270447144122 40.75373371438336...,11426
4,http://nyc.pediacities.com/Resource/PostalCode...,0,081,5,Fresh Meadows,NY,36,6.938587e+07,38693.565676,Queens,POLYGON ((-73.81088634744755 40.72717187575918...,11365


In [14]:
zipcodes = []
for i in range(len(geonyc)):
    zipcode = geonyc.ix[i]['postalCode']
    zipcodes.append(int(zipcode))

In [15]:
#change colum name and delet what we dont need
geonyc = geonyc.rename(columns={'postalCode': "zip"})
geonyc = geonyc.loc[:,['zip', 'geometry',]]

In [16]:
geonyc.head()

,zip,geometry
0,11372,POLYGON ((-73.86942457284175 40.74915687096787...
1,11004,POLYGON ((-73.71068374074007 40.75004039450917...
2,11040,POLYGON ((-73.70098278625547 40.73889569923034...
3,11426,POLYGON ((-73.72270447144122 40.75373371438336...
4,11365,POLYGON ((-73.81088634744755 40.72717187575918...


In [17]:
#cbd data
cbd = pd.DataFrame()
for yr in range(1994,2015):
    year = str(yr)[2:4]
    fname = 'zbp' + year + 'totals.zip'
    zf = zipfile.ZipFile(path + fname)
    df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    df['year'] = year
    temp = df.loc[:,['zip','year','est']]
    cbd = pd.concat([cbd, temp], axis=0)

In [18]:
cbd.head()

,zip,year,est
0,1001.0,94,439.0
1,1002.0,94,450.0
2,1003.0,94,10.0
3,1004.0,94,47.0
4,1005.0,94,92.0


In [19]:
cbd['zip'] = pd.to_numeric(cbd.zip).values
cbd['year'] = pd.to_numeric(cbd.year).values
cbd.index = cbd.zip
cbd.head()

,zip,year,est
zip,,,
1001.0,1001.0,94,439.0
1002.0,1002.0,94,450.0
1003.0,1003.0,94,10.0
1004.0,1004.0,94,47.0
1005.0,1005.0,94,92.0


In [26]:
cbddrop = cbd.ix[zipcodes].dropna(how='any')

cbddrop.reset_index(drop=True, inplace=True)
cbddrop.head()

,zip,year,est
0,11372.0,94.0,1229.0
1,11372.0,95.0,1236.0
2,11372.0,96.0,1292.0
3,11372.0,97.0,1309.0
4,11372.0,3.0,1521.0


### 2. merge data

In [27]:
# ZIP as index, YEAR as columns, and EST as values
cbd_pivot = cbd.pivot_table(index='zip', columns='year', values='est')
cbd_pivot.dropna(axis = 1)
cbd_pivot.head()

year,3,4,5,6,7,8,9,10,11,12,13,14,94,95,96,97
zip,,,,,,,,,,,,,,,,
501.0,7.0,8.0,2.0,8.0,5.0,3.0,2.0,2.0,2.0,2.0,2.0,4.0,NaN,2.0,4.0,4.0
1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,11.0,8.0
1001.0,505.0,500.0,509.0,509.0,503.0,477.0,458.0,448.0,436.0,453.0,439.0,458.0,439.0,439.0,433.0,457.0
1002.0,586.0,584.0,589.0,582.0,596.0,584.0,571.0,570.0,558.0,572.0,572.0,560.0,450.0,467.0,487.0,500.0
1003.0,16.0,15.0,12.0,12.0,16.0,17.0,16.0,16.0,16.0,15.0,15.0,18.0,10.0,10.0,13.0,15.0


In [28]:
cbd_pivot.shape

(42540, 16)

# mistakes..


### white data

In [23]:
cbd_standardized = cbd_pivot.dropna(how='any').copy()
for row in cbd_standardized.index:
    row_values = cbd_pivot.ix[row]
    row_mean = row_values.mean()
    row_std = row_values.std()
    cbd_standardized.ix[row] = (row_values - row_mean)/row_std

In [24]:
cbd_standardized.head()

year,3,4,5,6,7,8,9,10,11,12,13,14,94,95,96,97
zip,,,,,,,,,,,,,,,,
1001.0,1.323205,1.151639,1.460458,1.460458,1.254579,0.362434,-0.289518,-0.632651,-1.044410,-0.461084,-0.941470,-0.289518,-0.941470,-0.941470,-1.147350,-0.323831
1002.0,0.724192,0.681903,0.787624,0.639614,0.935634,0.681903,0.407027,0.385883,0.132152,0.428172,0.428172,0.174440,-2.151430,-1.791978,-1.369092,-1.094216
1003.0,0.626452,0.208817,-1.044086,-1.044086,0.626452,1.044086,0.626452,0.626452,0.626452,0.208817,0.208817,1.461721,-1.879355,-1.879355,-0.626452,0.208817
1004.0,-0.548714,1.160555,0.228226,0.150532,-0.626408,-0.781797,-0.781797,-0.548714,-0.704103,-0.937185,-0.937185,-1.170267,1.471332,1.549026,1.471332,1.005167
1005.0,-0.673044,0.171560,-0.461893,0.382711,2.071921,0.171560,-1.939951,-0.039591,-1.095347,-0.461893,0.171560,0.171560,-0.884195,-0.250742,1.016165,1.649618


In [25]:
def elbow(data, K):
    '''This function is to calculate average within-cluster
    sum of squares, and plot the "elbow"'''
    
    KM = [KMeans(n_clusters=k).fit(data) for k in K]
    centroids = [k.cluster_centers_ for k in KM]
    
    D_k = [cdist(data, cent, 'euclidean') for cent in centroids]
    globals()['dist'] = [np.min(D, axis=1) for D in D_k]
    avgWithinSS = [sum(d)/data.shape[0] for d in dist]
    
    # elbow curve
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, avgWithinSS, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Average within-cluster sum of squares')
    plt.title('Elbow for KMeans clustering')
    plt.show()

In [ ]:
elbow(cbd_standardized, range(1,30))

you may need to clean your data: for some NYC zip codes there may be no info
    
sanity check: you should have 20 (N_timestamps) datapoints per time series and about 250 zipcodes (Nzipcodes)

IMPORTANT: we talked about the importance of "whitening" your data: dividing each feature by its standard deviation. 
Whitenings decorrelates the data: it makes the features independent so that the data covariance matrix is the identity matrix.
Whitening your data in time series analysis is in most cases **wrong**: you are modifying your time behaviour. This is because of the strong correlation between features (two consecutive time stamps for the same observation, the same zip code here, are strongly correlated). Here instead you want to standardize your time series: subtract the mean and divide each time series (separately) by its standard deviation. As a sanity check (if you use skitlearn Kmeans or skitlearns kmeans2): you want your data array to be shaped Nzipcodes x Ntimestamps

mydata.shape should be (Nzipcodes, Ntimestamps)

mydata[i].std() shoould be 1 for all i in range(len(Nzipcodes))

mydata[i].mean() should be ~0 for all i in range(len(Nzipcodes))



TASKS:
    
    1. get and prep your data.
    2. cluster the NUMBER OF ESTABLISHMENTS time series with K-means in **a few** clusters (as discussed there is no real good, sound way to decide what a good number is here. try a few options, keeping in mind a few is more than a couple, but i recommand you stay within the single digit numbers)
    3. plot the cluster centers (if you used K means those are the means of the clusters). you can plot for example the cluster centers overlayed on each time series (using the alpha channel to control the opacity in the plot may be helpful here).
    4. Use another clustering algorithm (of your choice)
    5. overlay your data on a NYC map: you can use shapefiles for the zip codes and different colors for different clusters
    6. Compare the results of the 2 algorithms
    7. attempt an interpretation. this is dangerous ground: clustering is an exploratory tool so you do not want to jump to conclusions because you see some clusters! but seeing structure in your data can inform your next moves as an investigator. 
    

The map of the clusters may look something like this

In [ ]:
from PIL import Image
Image.open("clustermap.png")

And if you use hierarchical clustering and make a dandrogram it may look like this`

In [ ]:
from PIL import Image
Image.open("dandrogram.png")